# Running the Code

In this tutorial you will learn: 

1. How to run the code! 

You should already be familiar with: 

1. How to pick condensates to run in a cloud model 
2. What is $f_{sed}$ and what number is right for my model? 
3. What are the chemical limitations in the code 
4. How to compute initial Mie scattering grid


In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
from bokeh.palettes import Colorblind
output_notebook()
import numpy as np
import pandas as pd
import astropy.units as u
import justplotit

#here is pyeddy
import pyeddy as ed

Loading BokehJS ...

## Simple Isothermal Profile Example

Let's take the isothermal example from the first tutorial to begin gaining intuition for using the code

In [2]:
pressure = np.logspace(-5,3,30) #simple isotherml PT profile (kelvin)
temperature = np.zeros(30)+1600  
kz = np.zeros(30)+ 1e9 #constant kz profile

metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight

#get pyeddy recommendation for which gases to run
recommended_gases = ed.recommend_gas(pressure, temperature, metallicity,mean_molecular_weight)

print(recommended_gases)


['Cr', 'Fe', 'Mg2SiO4', 'MgSiO3']


/Users/nbatalh1/Documents/codes/pyeddy/root_functions.py:224: RuntimeWarning: divide by zero encountered in log10
  return np.log10(pv) - np.log10(partial_p)


### Define up the `Atmosphere Class` 

In [3]:
#set cloud species to condense, let's take 
#the recommended gases at face value for now
a = ed.Atmosphere(recommended_gases,fsed=0.1)

### Set gravity and the P/T/ Kz profile

In [4]:
#set the planet gravity
a.get_gravity(gravity=357.00, gravity_unit=u.Unit('cm/(s**2)'))

#PT 
a.get_pt(df = pd.DataFrame({'pressure':pressure, 'temperature':temperature,
                           'kz':kz}))

### Run the code 

In [5]:
#start with the simplest output 
#get total opacity, single scattering, asymmetry, and individual optical depths 
opd, w0, g0= ed.justdoit(a, directory='/Users/nbatalh1/Documents/data/eddy_optics')

In [6]:
#get full dictionary output 
all_out = ed.justdoit(a, as_dict=True, directory='/Users/nbatalh1/Documents/data/eddy_optics')

## Exploring `dict` Output

In [7]:
all_out.keys()

dict_keys(['pressure', 'pressure_unit', 'wave', 'wave_unit', 'condensate_mmr', 'cond_plus_gas_mmr', 'mean_particle_r', 'droplet_eff_r', 'r_units', 'column_density', 'column_density_unit', 'opd_per_layer', 'single_scattering', 'asymmetry', 'opd_by_gas'])

## Use `PICASO` to visuzalize output

`PICASO` has a number of functions to visualize cloud output. We can use some eddysed functions to get the output into the specific format. 

In [8]:
#get into DataFrame format
df = ed.picaso_format(opd,g0,w0)

In [9]:
import picaso.justplotit as jpi

In [10]:
show(jpi.plot_cld_input(nwno=196, nlayer=29,df=df,pressure=a.pressure))